# **PPW Crawling Berita**

Dibuat Oleh:

* Nama : Zainal Arifin
* NIM : 220411100103
* Kelas : PPW A

## **Pengertian Crawling**

Crawling adalah proses yang digunakan oleh search engine untuk menjelajahi atau merayapi website untuk mendapatkan informasi sebagai respons terhadap kueri yang diberikan. Proses ini dilakukan dengan bantuan bot yang dikenal dengan nama web crawler atau crawler. Crawler adalah mesin pencari bot yang khusus digunakan untuk mencari, mengunjungi, dan menganalisis konten situs web dan informasi lainnya di internet. Crawling data bertujuan untuk mengumpulkan data secara otomatis dari website dan membangun indeks data untuk digunakan mesin pencari.

### **Proses Crawling**

**Tool atau Library**

In [ ]:
!pip install selenium

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

* import pandas as pd: Untuk mengelola data dalam tabel.
* from selenium import webdriver: Untuk mengontrol browser otomatis.
* from selenium.webdriver.chrome.options import Options: Untuk mengatur pengaturan browser Chrome.
* from selenium.webdriver.common.by import By: Untuk mencari elemen di halaman web.
* from selenium.webdriver.support.ui import WebDriverWait: Untuk menunggu elemen muncul di halaman.
* from selenium.webdriver.support import expected_conditions as EC: Untuk memeriksa kondisi di halaman web.
* import time: Untuk menambahkan jeda waktu dalam kode.

In [ ]:
def web_driver():
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

Fungsi web_driver ini menyiapkan browser Chrome untuk berjalan di latar belakang tanpa tampilan grafis, serta mengatasi beberapa potensi masalah yang mungkin muncul pada sistem atau penggunaan memori. Dengan fungsi ini, kamu bisa menjalankan browser Chrome secara otomatis untuk keperluan seperti web scraping atau otomatisasi, tanpa perlu melihat tampilan browser di layar.

In [ ]:
def get_element_text(driver, xpath):
    try:
        return WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, xpath))
        ).text.strip()
    except Exception as e:
        print(f"Error finding element with XPath {xpath}: {e}")
        return ""

Mencari elemen di halaman web berdasarkan XPath, dan jika elemen ditemukan, teksnya akan diambil. Jika terjadi kesalahan, fungsi ini akan menangani kesalahan tersebut dengan mencetak pesan dan mengembalikan string kosong.

In [ ]:
def extract_article_content(driver, article_url):
    driver.get(article_url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//h1'))
    )
    title = get_element_text(driver, './/h1')

    date = get_element_text(driver, './/p[@class="pt-20 date"]')

    content_elements = driver.find_elements(By.XPATH, './/div[@class="news-text"]/p')
    content = " ".join(p.text for p in content_elements)

    return {
        "Title": title,
        "Date": date,
        "Content": content
    }

Membuka halaman Berita, menunggu elemen-elemen penting muncul, dan mengambil teks dari judul, tanggal, dan konten isi, kemudian mengembalikannya dalam format dictionary.

In [ ]:
def extract_category_data(driver):
    articles = driver.find_elements(By.XPATH, '//div[@class="item"]')
    article_urls = [article.find_element(By.XPATH, './/a').get_attribute('href') for article in articles]
    return article_urls

def scrape_category_page(driver, main_url, category_name, max_articles=50):
    all_article_data = []
    page_num = 1

    while len(all_article_data) < max_articles:
        current_url = f"{main_url}?page={page_num}"
        driver.get(current_url)

        article_urls = extract_category_data(driver)

        if not article_urls:
            print(f"No more articles found on page {page_num}. Stopping.")
            break

        for url in article_urls:
            if len(all_article_data) >= max_articles:
                break

            article_data = extract_article_content(driver, url)
            article_data["Category"] = category_name
            all_article_data.append(article_data)
        page_num += 1
        time.sleep(2)

    return all_article_data

driver = web_driver()

kategori_list = [
    {"url": "https://www.metrotvnews.com/channel/ekonomi", "name": "Ekonomi"},
    {"url": "https://www.metrotvnews.com/channel/olahraga", "name": "Olahraga"}
]

all_data = []

for category in kategori_list:
    print(f"Scraping category: {category['name']}")
    category_data = scrape_category_page(driver, category["url"], category["name"], max_articles=50)
    all_data.extend(category_data)

driver.quit()

Scraping category: Ekonomi
Scraping category: Olahraga


Mengakses beberapa halaman kategori berita, mengambil URL berita dari setiap halaman, dan kemudian mengambil konten dari berita-berita tersebut, sambil memastikan untuk tidak melebihi jumlah berita maksimal

In [ ]:
df = pd.DataFrame(all_data)
df.to_csv('hasil_crawling.csv', index=False)

In [ ]:
df

,Title,Date,Content,Category
0,PalmCo Dukung Cyber Education di Sekolah Remot...,Ade Hapsari Lestarini • 11 September 2024 08:59,Jakarta: Sub Holding Perkebunan Nusantara PTPN...,Ekonomi
1,Harga Jual Emas Antam Melesat Rp13 Ribu/Gram,Arif Wicaksono • 11 September 2024 08:55,Jakarta: Harga emas batangan PT Aneka Tambang ...,Ekonomi
2,Terpopuler Ekonomi: Pascamerger Angkasa Pura h...,Ade Hapsari Lestarini • 11 September 2024 08:28,Jakarta: Berita-berita terpopuler ekonomi pada...,Ekonomi
3,Inflasi dan Debat Presiden AS Bikin Harga Emas...,Ade Hapsari Lestarini • 11 September 2024 08:12,Jakarta: Harga emas dunia menguat pada sesi pe...,Ekonomi
4,"Harga Minyak Mentah Brent Terpental ke USD69,1...",Ade Hapsari Lestarini • 11 September 2024 07:57,New York: Harga minyak mentah dunia terpantau ...,Ekonomi
...,...,...,...,...
95,Indonesia Ditunjuk jadi Tuan Rumah World Abili...,Gregorius Gelino • 6 September 2024 12:07,Paris: Menteri Pemuda dan Olahraga Republik In...,Olahraga
96,Menpora Berharap Banyak dengan PON XXI Aceh-Sumut,Gregorius Gelino • 6 September 2024 11:19,Jakarta: Menteri Pemuda dan Olahraga Republik ...,Olahraga
97,IFG Labuan Bajo Marathon 2024 Siapkan Hadiah R...,Gregorius Gelino • 6 September 2024 10:25,Labuan Bajo: Pendaftaran IFG Labuan Bajo Marat...,Olahraga
98,Hasil Pertandingan UEFA Nations League,Alfa Mandalika • 6 September 2024 09:22,Jakarta: Sejumlah negara melakoni laga UEFA Na...,Olahraga
